In [1]:
import ee
import os
import logging
from src.utils.cloud_utils import download_gcs, ee_task_list_poller
from src.create_canopy_guide import cg
from src.calc_FM40 import fm40
from src.calc_CC_CH import cc_ch
from src.calc_CBD_CBH import cbd_cbh
from src.export_fuelstack import export_fuelstack
import time
import datetime

repo_dir =  os.path.abspath(os.path.join("__file__",".."))
date_id = datetime.datetime.utcnow().strftime("%Y-%m-%d").replace('-','') # like 20221216
logging.basicConfig(
    format="%(asctime)s %(message)s",
    datefmt="%Y-%m-%d %I:%M:%S %p",
    level=logging.WARNING,
    filename = os.path.join(repo_dir,"log",f"{date_id}.log")
)
logger = logging.getLogger(__name__)

logger.setLevel(logging.INFO)

ee.Initialize(project='pyregence-ee')

%load_ext autoreload
%autoreload 2

### Make a folder to hold fuel layers generated from your treatment scenario ensemble

In [2]:
scenarios_path = "projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_noPriority_20230217/treatment_scenarios" # change this to given runs folder path
scenarios = ee.ImageCollection(scenarios_path)
#create a fuelscapes folder within the runs folder
fuels_folder=scenarios_path.replace('treatment_scenarios','fuelscape_scenarios')
fuels_folder_list = os.popen(f"earthengine ls {os.path.dirname(scenarios_path)}").read().split('\n')[0:-1]
if not fuels_folder in fuels_folder_list:
    os.popen(f"earthengine create folder {fuels_folder}").read()
    print(f'Created Folder: {fuels_folder}')
else:
    print(f"{fuels_folder} already exists")

projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_noPriority_20230217/fuelscape_scenarios already exists


### For each treated landscape img in your treatment scenarios imageCollection, create a new folder to hold the fuels layers

In [3]:
scenarios_img_paths = os.popen(f"earthengine ls {scenarios_path}").read().split('\n')[0:-1]
scn_sub_folders = [(fuels_folder+'/'+os.path.basename(path)) for path in scenarios_img_paths]

sub_folders_list = os.popen(f"earthengine ls {fuels_folder}").read().split('\n')[0:-1]
for sub_folder_name in scn_sub_folders:
    if not sub_folder_name in sub_folders_list:
        os.popen(f"earthengine create folder {sub_folder_name}").read()
        print(f'Created scenario fuelscape sub-folder: {sub_folder_name}')
    else:
        print(f"{sub_folder_name} already exists")

projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_noPriority_20230217/fuelscape_scenarios/scenario1 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_noPriority_20230217/fuelscape_scenarios/scenario10 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_noPriority_20230217/fuelscape_scenarios/scenario11 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_noPriority_20230217/fuelscape_scenarios/scenario12 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_noPriority_20230217/fuelscape_scenarios/scenario13 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_noPriority_20230217/fuelscape_scenarios/scenario2 already exists
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_noPriority_20230217/fuelscape_scenarios/scenario3 already exists
projects/pyregence-ee/a

### Loop thru each fuel updater script the way we do for FireFactor: create_canopy_guide, calc_CC_CH, calc_CBD_CBH, calc_FM40 (don't need to run fm40 wui update)

### Export Canopy Guide and FM40

In [ ]:
# Create Canopy Guide
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    cg_ic = cg(dist_img_path=scn_img_path) # don't want to poll each submitted task before launching next..
    #break

# how to check they are all done in a while loop? we've got n scns X n lf zone exports 

# for scn in cg_ic scns:
#   while scn.size().. (but we don't know what .size() should be inherently, could return len() of export tasks generated within cg function.

In [ ]:
# Create FM40 
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    fm40_ic = fm40(dist_img_path = scn_img_path, fuels_source='firefactor')
    #break
# same polling idea applies as in CG..

### Export CC and CH (wait until all Canopy Guide exports have successfully completed before running)

In [ ]:
# CC and CH
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    cc,ch = cc_ch(dist_img_path=scn_img_path,fuels_source='firefactor')
    # poll in the last iteration of loop? exactly two tasks each iteration but don't want to wait for each scns tasks to complete in case more than 2 tasks can run concurrently on server
    break

### Export CBD and CBH (wait until all CC and CH exports have completed before running)

In [ ]:
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    cbd,cbh = cbd_cbh(dist_img_path=scn_img_path,fuels_source='firefactor')
    # poll in last iteration of loop? exactly two tasks each iteration
    break

### After all fuel layers in each scenario's fuelscape has successfully exported, collate each 5-layer fuelscape into a multiband img for Google Drive Export

In [5]:
for sub_folder_name in scn_sub_folders:
    print(sub_folder_name)
    gcs_fuelstack = "gs://sm-tx/runs_scenarios_13_distro_norm_noPriority_20230217/fuelscape_scenarios/scenario1.tif"
    # gcs_fuelstack = export_fuelstack(folder=sub_folder_name,poll=True)
    print(gcs_fuelstack)
    local_fuelstack = download_gcs(gcs_fuelstack)
    break

projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_noPriority_20230217/fuelscape_scenarios/scenario1
gs://sm-tx/runs_scenarios_13_distro_norm_noPriority_20230217/fuelscape_scenarios/scenario1.tif
